In [8]:
import os
import dotenv
dotenv.load_dotenv(".env.local")

from supabase import create_client, Client
import time
from copy import deepcopy

import re
import json
from urllib.request import urlopen
import requests
import pandas as pd
import plotly.express as px
px.set_mapbox_access_token(os.environ["MAPBOX_TOKEN"])

In [46]:
client = create_client("https://wvklzyidkkrskdgquocj.supabase.co", os.environ["SUPABASE_KEY"])
response = client.table('visits').select("*").execute()

2024-03-01 16:42:36,494:INFO - HTTP Request: GET https://wvklzyidkkrskdgquocj.supabase.co/rest/v1/visits?select=%2A "HTTP/1.1 200 OK"


In [47]:
visits = deepcopy(response.data)
visits_df = pd.DataFrame(visits)
visits_df['created_at'] = pd.to_datetime(visits_df['created_at'])
visits_df

,id,created_at,ip_address,route,user_agent,is_bot,referrer
0,7,2024-02-27 04:41:20.067741+00:00,64.23.187.48,/,None,None,None
1,8,2024-02-27 04:41:20.795991+00:00,64.23.187.48,/,None,None,None
2,9,2024-02-27 04:41:21.213661+00:00,172.69.23.187,/,None,None,None
3,10,2024-02-27 04:42:53.205514+00:00,76.99.220.36,/,None,None,None
4,11,2024-02-27 04:43:51.899723+00:00,172.69.22.42,/,None,None,None
...,...,...,...,...,...,...,...
329,31,2024-02-27 04:58:16.901783+00:00,127.0.0.1,/technology/engcode,None,None,None
330,157,2024-02-29 01:09:56.447413+00:00,127.0.0.1,/,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,false,None
331,158,2024-02-29 01:12:02.064846+00:00,127.0.0.1,/technology,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,false,None
332,159,2024-02-29 01:12:07.331389+00:00,127.0.0.1,/technology/seeing-sound,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7...,false,None


In [48]:
maximum_session_gap = 30
sessions = []
for ip_address in visits_df['ip_address'].unique():
    visitors_visits = visits_df[visits_df['ip_address'] == ip_address]
    visitors_visits = visitors_visits.sort_values('created_at')
    session = []
    for i in range(1, len(visitors_visits)):
        if session == []:
            session.append(visitors_visits.iloc[i])
        previous_visit = session[-1]
        current_visit = visitors_visits.iloc[i]
        time_difference = (current_visit['created_at'] - previous_visit['created_at']).seconds / 60
        if time_difference > maximum_session_gap:
            sessions.append(session)
            session = []
        session.append(current_visit)
    if session != []:
        sessions.append(session)

sessions_df = {
    "ip_address": [],
    "start_time": [],
    "end_time": [],
    "visit_ids": []
}

for session in sessions:
    sessions_df["ip_address"].append(session[0]['ip_address'])
    sessions_df["start_time"].append(session[0]['created_at'])
    sessions_df["end_time"].append(session[-1]['created_at'])
    sessions_df["visit_ids"].append([visit['id'] for visit in session])

sessions_df = pd.DataFrame(sessions_df)

In [73]:
response = client.table('visitors').select("*").execute()
visitors = deepcopy(response.data)

2024-03-01 17:01:37,400:INFO - HTTP Request: GET https://wvklzyidkkrskdgquocj.supabase.co/rest/v1/visitors?select=%2A "HTTP/1.1 200 OK"


In [91]:
visitors_df = pd.DataFrame(visitors)
new_visitors = []
ip_addresses_in_sessions = sessions_df['ip_address'].unique()
for ip_address in ip_addresses_in_sessions:
    if ip_address in visitors_df['ip_address'].values:
        continue
    ip_info = json.load(urlopen(f"https://ipinfo.io/{ip_address}/json"))
    loc = ip_info.get('loc', None)
    if loc:
        lat, lon = loc.split(',')
        ip_info['lat'] = float(lat)
        ip_info['lon'] = float(lon)
    else:
        ip_info['lat'] = None
        ip_info['lon'] = None
    visitor = {
        "ip_address": ip_address,
        "city": ip_info.get('city', None),
        "region": ip_info.get('region', None),
        "country": ip_info.get('country', None),
        "postal": ip_info.get('postal', None),
        "lat": ip_info.get('lat', None),
        "lon": ip_info.get('lon', None),
        "timezone": ip_info.get('timezone', None),
        "hostname": ip_info.get('hostname', None),
        "org": ip_info.get('org', None),
        "is_bot": visits_df[visits_df['ip_address'] == ip_address].iloc[-1]['is_bot']
    }
    new_visitors.append(visitor)

response = client.table('visitors').insert(new_visitors).execute()

2024-03-01 17:19:19,261:INFO - HTTP Request: POST https://wvklzyidkkrskdgquocj.supabase.co/rest/v1/visitors "HTTP/1.1 201 Created"


In [103]:
fig = px.scatter_mapbox(
    session_df, 
    lat="lat", 
    lon="lon", 
    color="end_time", 
    text="city", 
    size=[25] * len(session_df),
    center={"lat": session_df.lat.median(), "lon": session_df.lon.median()},
    zoom=1
)
fig.show()